In [0]:
# Binary classification code:
!wget --no-check-certificate \                                       # Download the dataset
    https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip \
    -O /tmp/cats_and_dogs_filtered.zip

--2019-07-06 05:50:23--  https://storage.googleapis.com/mledu-datasets/cats_and_dogs_filtered.zip
Resolving storage.googleapis.com (storage.googleapis.com)... 74.125.142.128, 2607:f8b0:400e:c02::80
Connecting to storage.googleapis.com (storage.googleapis.com)|74.125.142.128|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 68606236 (65M) [application/zip]
Saving to: ‘/tmp/cats_and_dogs_filtered.zip’

/tmp/cats_and_dogs_ 100%[===================>]  65.43M   139MB/s    in 0.5s    

2019-07-06 05:50:29 (139 MB/s) - ‘/tmp/cats_and_dogs_filtered.zip’ saved [68606236/68606236]



In [0]:
#This is part is optional-----------------------------
import os
import zipfile

file = "/tmp/cats_and_dogs_filtered.zip"
zf = zipfile.ZipFile(file, 'r')
zf.extractall('/tmp/cats_and_dogs_filtered')
zf.close()
#-------------------------------------------------------

In [16]:
#You can start from here
#Import the dataset for training the model........................................
path1 = str(input("Enter the path to directory containing image dataset(Folder name must be named after image name): "))
train_dst = os.path.join(path1)
groups = os.listdir(train_dst)
if(len(groups) !=2):
  exit(-1)

Enter the path to directory containing image dataset(Folder name must be named after image name): /tmp/cats_and_dogs_filtered/cats_and_dogs_filtered/train


In [17]:
#For checking the model performance and increase it.......................................
dec = input("Do you have validation data?(y/n): ")
if(dec == 'y' or dec == 'Y'):
  path2 = input("Enter the path to directory containing validation image dataset(Folder name must be named after image name): ")
  validation_dst = os.path.join(path2)
  if(len(os.listdir(validation_dst)) !=2):
    exit(-1)

Do you have validation data?(y/n): y
Enter the path to directory containing validation image dataset(Folder name must be named after image name): /tmp/cats_and_dogs_filtered/cats_and_dogs_filtered/validation


In [20]:
print("Total Training Dataset: ")
print(groups[0] + " = " + str(len(os.listdir(train_dst + "/" + groups[0]))))
print(groups[1] + " = " + str(len(os.listdir(train_dst + "/" + groups[1]))))
print("Total Validation Dataset:")
print(groups[0] + " = " + str(len(os.listdir(validation_dst + "/" + groups[0]))))
print(groups[1] + " = " + str(len(os.listdir(validation_dst + "/" + groups[1]))))
print()

Total Training Dataset: 
dogs = 1000
cats = 1000
Total Validation Dataset:
dogs = 500
cats = 500



In [21]:
import tensorflow as tf

model = tf.keras.models.Sequential([tf.keras.layers.Conv2D(16,(3,3),activation='relu',input_shape=(300,300,3)),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(32,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Conv2D(64,(3,3),activation='relu'),
                                    tf.keras.layers.MaxPooling2D(2,2),
                                    tf.keras.layers.Flatten(),
                                    tf.keras.layers.Dense(512,activation='relu'),
                                    tf.keras.layers.Dense(1, activation='sigmoid')
                                   ])

model.summary()

#RMSprop is efficient gradient descent algorithm
from tensorflow.keras.optimizers import RMSprop
model.compile(loss='binary_crossentropy', optimizer=RMSprop(lr=0.001),metrics=['acc'])

#Preprocessing of images
from tensorflow.keras.preprocessing.image import ImageDataGenerator
# All images will be rescaled by 1./255
trainDataGen = ImageDataGenerator(rescale=1/255)
validationDataGen = ImageDataGenerator(rescale=1/255)
train_generator = trainDataGen.flow_from_directory(
                    path1,
                    target_size=(300,300),
                    batch_size=128,
                    class_mode='binary'    # Since we use binary_crossentropy loss, we need binary labels
                  )

validation_generator = validationDataGen.flow_from_directory(
                    path2,
                    target_size=(300,300),
                    batch_size=32,
                    class_mode='binary'    # Since we use binary_crossentropy loss, we need binary labels
                  )

history = model.fit_generator(train_generator,
                              steps_per_epoch=8,
                              epochs=15,
                              verbose=1,
                              validation_data=validation_generator,
                              validation_steps=8
                             )


Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_5 (Conv2D)            (None, 298, 298, 16)      448       
_________________________________________________________________
max_pooling2d_5 (MaxPooling2 (None, 149, 149, 16)      0         
_________________________________________________________________
conv2d_6 (Conv2D)            (None, 147, 147, 32)      4640      
_________________________________________________________________
max_pooling2d_6 (MaxPooling2 (None, 73, 73, 32)        0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 71, 71, 64)        18496     
_________________________________________________________________
max_pooling2d_7 (MaxPooling2 (None, 35, 35, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 33, 33, 64)       

In [24]:
import numpy as np
from keras.preprocessing import image
file = input("Enter an image to test: ")
x = image.load_img(file,target_size=(300,300))   #loading image of size 300x300
x = image.img_to_array(x)                   #loading
x = np.expand_dims(x,axis=0)
images = np.vstack([x])
classes = model.predict(images, batch_size=10)

if classes[0]>0.5:
   print(file + ' is ' + groups[0] + ' image')
else:
    print(file + ' is ' + groups[1] + ' image')

Enter an image to test: /dog.jpg
/dog.jpg is dogs image
